In [14]:
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from utils_mitgcm import open_mitgcm_ds_from_config

In [3]:
model = 'geneva_200m'

In [4]:
outputpath = "../99-Outputs/nencioli/"

# Open MITgcm results

In [5]:
mitgcm_config, ds_mitgcm = open_mitgcm_ds_from_config('../config.json', model)

In [6]:
temp_ini = ds_mitgcm.THETA.isel(time=0,Z=0)
mask = temp_ini.where(abs(temp_ini) > 1e-10).values

# Open eddy dataset

In [7]:
eddies_nencioli_path = r'../99-Outputs/nencioli/winding_criteria_eddies.nc'
xr_eddies_nencioli = xr.open_dataset(eddies_nencioli_path)

In [8]:
eddies_swirl_path = r'../99-Outputs/swirl/200m_eddies.nc'
xr_eddies_swirl = xr.open_dataset(eddies_swirl_path)

# Plot

In [9]:
def plot_stream_velocity(ax, snapshot_vel, stream_density = 6):
    x_plot, y_plot = np.meshgrid(snapshot_vel['XC'], snapshot_vel['YC'])
    u_plot = snapshot_vel['UVEL'].values
    v_plot = snapshot_vel['VVEL'].values

    # Plot streamlines
    ax.streamplot(x_plot, y_plot, u_plot, v_plot, 
                   density=stream_density, color='black', linewidth=0.5, 
                   arrowsize=0.7, arrowstyle='->')

In [22]:
def plot_snapshot(t_index, ds_mitgcm, xr_eddies_nencioli, xr_eddies_swirl):
    time = ds_mitgcm.isel(time=t_index).time.values
    
    fig = plt.figure(figsize=(20,7))
    snapshot_vel = ds_mitgcm.isel(time=t_index, Z=0)
    x_plot, y_plot = np.meshgrid(snapshot_vel['XC'], snapshot_vel['YC'])
    u_plot = snapshot_vel['UVEL'].values
    v_plot = snapshot_vel['VVEL'].values
    
    snapshot_vel.UVEL.plot()
    # Plot streamlines
    plt.streamplot(x_plot, y_plot, u_plot, v_plot, 
                   density=10, color='black', linewidth=0.5, 
                   arrowsize=0.7, arrowstyle='->')
    
    snapshot_nencioli = xr_eddies_nencioli.sel(eddy_id=xr_eddies_nencioli.time == time)
    snapshot_nencioli = snapshot_nencioli.sel(eddy_id=snapshot_nencioli.valid_eddy == 1)
    plt.scatter(snapshot_nencioli.lon.values, 
               snapshot_nencioli.lat.values, 
               color='black', 
               edgecolors='black',
               linewidth=1,
               marker='o',
               s=50,
               zorder=10)
    
    snapshot_swirl = xr_eddies_swirl.sel(eddy_id=xr_eddies_swirl.time == time)
    colors_swirl = ['gold' if t == -1 else 'green' for t in snapshot_swirl.type.values]
    plt.scatter(snapshot_swirl.x_center.values, 
               snapshot_swirl.y_center.values, 
               color=colors_swirl, 
               edgecolors='black',
               linewidth=1,
               marker='^',
               s=50,
               zorder=10)
    plt.title(time)
    
    nencioli_handle = Line2D([], [], marker='o', color='black', markersize=8, linestyle='None',
                                label='Nencioli')
    swirl_handle = Line2D([], [], marker='^', color='black', markersize=8, linestyle='None',
                            label='SWIRL')
    
    # Add to legend
    plt.legend(handles=[nencioli_handle, swirl_handle], loc='upper right')
    
    return fig

In [23]:
for t in range(len(ds_mitgcm.time.values)):
    fig = plot_snapshot(t,ds_mitgcm,xr_eddies_nencioli,xr_eddies_swirl)
    plt.savefig(rf'../99-Outputs/comparison/nencioli_vs_swirl/t{t}')
    plt.close('all')